# Progetto ML: Classificazione Pezzi Difettosi - AutomaParts S.p.A.

**Obiettivo:** Creare un modello per individuare i pezzi difettosi nella linea di produzione (0 = conforme, 1 = difettoso). Permettendo: 

+ scarto automatico o blocco in ispezione finale;
+ instradamento verso un controllo 100% quando la probabilità è incerta;
+ analisi delle cause principali dei difetti per interventi di processo.

Le metriche chiave saranno quelle legate alla capacità di trovare i difetti (Recall) senza scartare troppi pezzi buoni.

In [ ]:
# Importo tutte le librerie

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_curve, auc
)

## 1. Caricamento Dati
Carico il dataset e do un'occhiata veloce.

In [ ]:
df = pd.read_csv('parts_production_data.csv')
df.head()

In [ ]:
print(df.shape)
df.info()

Nota: `production_timestamp` è object, dovrò convertirlo. Anche `material_batch` è una stringa.

In [ ]:
df.describe()
# Qui do un occhiata ai dati per medie e outlier

## 2. Analisi Esplorativa (EDA)
Controllo quanti difetti ci sono rispetto ai pezzi ok.

In [ ]:
print(df['defect_label'].value_counts())

sns.countplot(x='defect_label', data=df)
plt.title('Distribuzione Classi')
plt.show()

C'è un evidente sbilanciamento. I difetti sono molti meno dei pezzi conformi, il che è normale, ma dovrò tenerne conto.

Vediamo se i difetti dipendono dalla linea o dalla stazione.

In [ ]:
# Visualizzazione Difetti per Linea e Stazione (uso queste perché hanno dati fissi, bassa cardinalità)
plt.figure(figsize=(10, 5))
sns.countplot(x="line_id", hue="defect_label", data=df)
plt.title("Difetti per Linea")
plt.show()

plt.figure(figsize=(14, 5))
sns.countplot(x="station_id", hue="defect_label", data=df)
plt.title("Difetti per Stazione")
plt.show()

Sembra tutto abbastanza uniforme, non vedo picchi strani su linee specifiche.

### Correlazioni
La Matrice di correlazione mostra subito se ci sono correlazioni tra le features e la variabile target (difettosità).

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df.corr(numeric_only=True), annot=True, fmt=".2f", cmap='coolwarm')
plt.show()

## 3. Pulizia e Feature Engineering
Controllo valori nulli.

In [ ]:
df.isnull().sum()

Nessun valore nullo, ottimo.

### Gestione Outlier
Provo a togliere gli outlier estremi con IQR sulle misure principali.

In [ ]:
def clean_outliers(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return df[(df[col] >= lower) & (df[col] <= upper)]

cols = ['measure_diam_mm', 'measure_length_mm', 'temp_process_C', 'vibration_level']
print(f"Righe iniziali: {len(df)}")

for c in cols:
    df = clean_outliers(df, c)

print(f"Righe finali: {len(df)}")

Nessun outlier rimosso. Procedo con la pulizia dei dati (drop variabili inutili) e feature engineering.

Droppo  e  (inutili/rumore). Estraggo l'ora dal timestamp e poi rimuovo l'originale.

In [ ]:
# Droppo le colonne inutili o che non servono al modello
df = df.drop(columns=['part_id', 'operator_id'], errors='ignore')

if 'production_timestamp' in df.columns:
    df['production_timestamp'] = pd.to_datetime(df['production_timestamp'])
    df['hour'] = df['production_timestamp'].dt.hour
    df = df.drop(columns=['production_timestamp'], errors='ignore')

df.head()

### Material Batch
Cerco di estrarre informazioni utili dal codice `material_batch` (Anno, Settimana, Sequenza).

In [ ]:
def parse_batch(s):
    # es. MB-2024W24-L02-575
    try:
        parts = s.split('-')
        year_wk = parts[1].split('W')
        return int(year_wk[0]), int(year_wk[1]), int(parts[3]) # Anno, Week, Seq
    except:
        return None, None, None

batch_infos = df['material_batch'].apply(parse_batch)

df['batch_year'] = batch_infos.apply(lambda x: x[0])
df['batch_week'] = batch_infos.apply(lambda x: x[1])
df['batch_seq'] = batch_infos.apply(lambda x: x[2])

# Trasformazione ciclica per la settimana, così il modello capisce che la settimana 1 è vicina alla settimana 52 ecc.
df['batch_week_sin'] = np.sin(2 * np.pi * df['batch_week'] / 53)
df['batch_week_cos'] = np.cos(2 * np.pi * df['batch_week'] / 53)

df.head()

### Encoding
Uso LabelEncoder per le variabili categoriche rimaste (`material_batch`, `line_id`...).

In [ ]:
le = LabelEncoder()
cat_cols = df.select_dtypes(include=['object', 'string']).columns

for c in cat_cols:
    df[c] = le.fit_transform(df[c].astype(str))
    print(f"Encoded {c}")

df.head()

Aggiungo feature sulle deviazioni dalla media del lotto (magari aiuta a trovare anomalie locali).

In [ ]:
# Scostamento dal diametro medio del batch
df['batch_diam_mean'] = df.groupby('material_batch')['measure_diam_mm'].transform('mean')
df['diam_dev'] = df['measure_diam_mm'] - df['batch_diam_mean']

# Scostamento dalla temperatura media del batch
df['batch_temp_mean'] = df.groupby('material_batch')['temp_process_C'].transform('mean')
df['temp_dev'] = df['temp_process_C'] - df['batch_temp_mean']

df[['material_batch', 'diam_dev', 'temp_dev']].head()

Nota: vedo molti zeri, probabilmente i batch sono molto piccoli o quasi unici. Comunque le lascio.

## 4. Train-Test Split e Scaling

In [ ]:
X = df.drop('defect_label', axis=1)
y = df['defect_label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creaiamo un dataset per la LR con i dati scalati altrimenti il modello non lavora bene, per gli altri lascio i dati senza scalare
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Train shape: {X_train.shape}")
print(f"Test shape: {X_test.shape}")

## 5. Modelli (Baseline)
Provo 3 modelli: Logistic Regression, Decision Tree, Random Forest.
Uso i dati scalati per la LR, gli originali per gli alberi.

In [ ]:
# Logistic Regression
lr = LogisticRegression(random_state=42)
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)
print(f"LR Accuracy: {accuracy_score(y_test, y_pred_lr):.4f}")

# Decision Tree
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)
print(f"DT Accuracy: {accuracy_score(y_test, y_pred_dt):.4f}")

# Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print(f"RF Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")

La Random Forest sembra la migliore come accuracy, ma attenzione allo sbilanciamento.

## 6. Modelli Bilanciati (Class Weight)
Provo a usare `class_weight='balanced'` per aiutare i modelli a vedere meglio la classe minoritaria (i difetti).

In [ ]:
lr_bal = LogisticRegression(class_weight='balanced', random_state=42)
lr_bal.fit(X_train_scaled, y_train)

dt_bal = DecisionTreeClassifier(class_weight='balanced', random_state=42)
dt_bal.fit(X_train, y_train)

rf_bal = RandomForestClassifier(class_weight='balanced', random_state=42)
rf_bal.fit(X_train, y_train)

print("Training completato con class_weight='balanced'.")

## 7. Valutazione e Confronto
Confronto le metriche. Mi interessa soprattutto la **Recall** (non voglio perdermi pezzi difettosi).

In [ ]:
preds = {
    'LR Base': y_pred_lr,
    'DT Base': y_pred_dt,
    'RF Base': y_pred_rf,
    'LR Balanced': lr_bal.predict(X_test_scaled),
    'DT Balanced': dt_bal.predict(X_test),
    'RF Balanced': rf_bal.predict(X_test)
}

for name, p in preds.items():
    print(f"--- {name} ---")
    print(classification_report(y_test, p, zero_division=0))
    print("-"*30)

**Osservazioni:**
Industialmente è più importante trovare i difetti (Recall) che avere una buona precisione. 
- I modelli base faticano molto sulla classe 1 (difetti).
- `LR Balanced` ha la Recall migliore (trova circa metà dei difetti), ma paga molto in precisione.
- `RF Balanced` è molto precisa ma ha una Recall bassa (troppo prudente).
- `DT Balanced` è un buon compromesso intermedio.

### Matrici di Confusione

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()

for i, (name, p) in enumerate(preds.items()):
    cm = confusion_matrix(y_test, p)
    sns.heatmap(cm, annot=True, fmt='d', ax=axes[i], cmap='Blues')
    axes[i].set_title(name)

plt.tight_layout()
plt.show()

Praticamente i modelli "base" sono inutili: dicono sempre che il pezzo è buono per andare sul sicuro e si perdono quasi tutti i difetti.

Quelli "balanced" ci provano di più: la Logistic Regression ne becca la metà (74), ma vede difetti ovunque e dà un sacco di falsi allarmi. La Random Forest invece resta troppo "timida" e non si sbilancia mai. Mi sa che con questi dati è un bel casino distinguerli!

### Cross-Validation
Verifico che i risultati siano stabili con una 5-fold CV: è un trucco per capire se il modello funziona davvero o se ha avuto solo "fortuna" con la divisione dei dati.

In [ ]:
print("CV su Modelli Bilanciati (Metric: Recall)")
print("LR Balanced:", cross_val_score(lr_bal, X_train_scaled, y_train, cv=5, scoring='recall').mean())
print("DT Balanced:", cross_val_score(dt_bal, X_train, y_train, cv=5, scoring='recall').mean())
print("RF Balanced:", cross_val_score(rf_bal, X_train, y_train, cv=5, scoring='recall').mean())

La Logistic Regression è l'unica che ci sta provando davvero, beccando almeno la metà dei difetti (51%). Il Decision Tree cala brutto e la Random Forest è un disastro: con un 4% di recall vuol dire che si perde praticamente tutto.

Morale: qui il modello più semplice macina meglio di quelli complessi, che probabilmente non trovano un pattern chiaro e restano troppo cauti!

### Feature Importance (Random Forest)
Vediamo quali sono le variabili che il modello ritiene più utili.

In [ ]:
imps = rf_bal.feature_importances_
indices = np.argsort(imps)[::-1]

plt.figure(figsize=(10, 5))
plt.bar(range(X.shape[1]), imps[indices])
plt.xticks(range(X.shape[1]), X.columns[indices], rotation=90)
plt.title("Feature Importance (RF Balanced)")
plt.show()

Il tempo di ciclo e le vibrazioni sono le variabili che "pesano" di più per il modello. Invece quelle feature sulle deviazioni che abbiamo aggiunto noi sono in fondo alla classifica: non contano praticamente nulla, un flop totale!

## Conclusioni

Il problema principale è la scarsa capacità dei modelli di separare nettamente le classi, probabilmente perché i difetti non dipendono in modo semplice dalle misure disponibili.

1. **Se la priorità è non far passare difetti**: Usare **Logistic Regression Balanced** (Recall ~50%), mettendo in conto molto scarto di pezzi buoni da ricontrollare.
2. **Se si cerca un equilibrio**: Il **Decision Tree Balanced** offre un compromesso ragionevole.
3. **Random Forest** è troppo conservativa in questo caso specifico, tende a non segnalare difetti se non è sicura.

**Next steps:**
- Raccogliere più dati sui difetti.
- Indagare nuove feature (magari dati grezzi dai sensori invece che medie).